<a href="https://colab.research.google.com/github/omid-sakaki-ghazvini/Projects/blob/main/persian_car_plate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install Dependencies and Setup

<div style="direction:rtl">
<font color='green' size="5px">
 کتابخانه های مورد نیاز را نصب میکنیم
    </font>
</div>

In [ ]:
pip install -q ultralytics
pip install -q wandb

In [ ]:
import wandb
wandb.init(mode="disabled")

import os                        # To work with operation system commands
import cv2                       # To process images
import random                    # to generate random choices
import warnings                  # To avoid python warnings
import numpy as np               # To work with arrays
import pandas as pd              # To work with DataFrames
import seaborn as sns            # To visualizations
from tqdm import tqdm            # To represent progress bars
from ultralytics import YOLO     # To Create Yolo model
from termcolor import colored    # To colorfull outputs
import matplotlib.pyplot as plt  # To visualizations

# ignore python warnings
warnings.filterwarnings('ignore')

# add style to plots
plt.style.use('fivethirtyeight')
sns.set_style('darkgrid')

# 2. Load Data

## https://www.kaggle.com/datasets/omidsakaki1370/persian-car-plate

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omidsakaki1370/persian-car-plate")

print("Path to dataset files:", path)

<div style="direction:rtl">
<font color='green' size="5px">
توسط خط فرمان زیر، دیتا را فراخوانی میکنیم
    </font>
</div>

In [ ]:
# Path of car-detection dataset
plate_base = '/root/.cache/kagglehub/datasets/omidsakaki1370/persian-car-plate/versions/1'

# Path of yaml file
plate_yaml_path = os.path.join(plate_base, 'data.yaml')

# Path of Train directory
plate_train_dir = os.path.join(plate_base, 'train')
plate_train_images = os.path.join(plate_train_dir, 'images')
plate_train_labels = os.path.join(plate_train_dir, 'labels')

# Path of Validation directory
plate_valid_dir = os.path.join(plate_base, 'valid')
plate_valid_images = os.path.join(plate_valid_dir, 'images')
plate_valid_labels = os.path.join(plate_valid_images, 'labels')

# Path of Test directory
plate_test_dir = os.path.join(plate_base, 'test')
plate_test_images = os.path.join(plate_test_dir, 'images')
plate_test_labels = os.path.join(plate_test_dir, 'labels')

In [ ]:
print(colored(f' Number of Train Images : {len(os.listdir(plate_train_images))} ', 'blue', attrs=['bold']))
print(colored(f' Number of Validation Images : {len(os.listdir(plate_valid_images))}', 'blue', attrs=['bold']))
print(colored(f' Number of Test Images : {len(os.listdir(plate_test_images))} ', 'blue', attrs=['bold']))
print('_'*40)
print(colored(f' Number of All Images : {len(os.listdir(plate_train_images)) + len(os.listdir(plate_valid_images)) + len(os.listdir(plate_test_images))} ', 'green', attrs=['bold']))

In [ ]:
def Image_Size(train_images) :
    '''
    Read images and save image-sizes in a set
    '''
    img_sizes = set()

    # Loop over train images and read them
    for img_name in tqdm(os.listdir(train_images)) :
        img_path = os.path.join(train_images, img_name)
        img = cv2.imread(img_path)
        h, w, c = img.shape
        img_sizes.add((h, w))

    return img_sizes

img_sizes = Image_Size(plate_train_images)

if len(img_sizes) == 1 :
    print(colored(f'\n All the images have size of {img_sizes} ', 'green', attrs=['bold']))
else :
    print(colored(f'\n There are {len(img_sizes)} different image sizes ! ', 'red', attrs=['bold']))

In [ ]:
# Height and Width of all images
H = list(img_sizes)[0][0]
W = list(img_sizes)[0][1]

print(colored(f'Height : {H}\nWidth : {W}', 'black', attrs=['bold']))

## 3.Plot Random Images With BBOXs

In [ ]:
def TXT_To_BBOX(lbl_path) :
    '''
    Read txt files and extract points of bonding-boxes.
    '''
    # Read text file and split it by white-space
    bbox = open(lbl_path, 'r')
    bbox = bbox.read()
    bbox = bbox.split(' ')[1:]

    # determine points of bboxes edge
    p1 = [int(float(bbox[0])*H), int(float(bbox[1])*W)]
    p2 = [int(float(bbox[2])*H), int(float(bbox[3])*W)]
    p3 = [int(float(bbox[4])*H), int(float(bbox[5])*W)]
    p4 = [int(float(bbox[6])*H), int(float(bbox[7])*W)]

    # convert points to array and reshape it
    pts = np.array([p1, p2, p3, p4], np.int32)
    pts = pts.reshape((-1, 1, 2))
    return pts

In [ ]:
def JPG_Name_To_TXT(img_name) :
    '''
    Based on image name, find text file in label folder.
    '''
    # Seperate prefix and sufix of image name
    name = img_name.split('.')[:-1]
    name = '.'.join(name)
    txt_name = name + '.txt'
    return txt_name

In [ ]:
def Plot_Random_Images() :
    '''
    Read and randomly choose some images and plot them with their bboxes in train folder.
    '''
    # Select K random number of train images
    random_images = random.choices(os.listdir(plate_train_images), k=12)
    # manual iterator
    i = 1
    # plot images and bboxes
    plt.figure(figsize=(20, 7))
    for img_name in random_images :
        plt.subplot(2, 6, i)
        # Image
        img_path = os.path.join(plate_train_images, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # BBox
        txt_name = JPG_Name_To_TXT(img_name)
        lbl_path = os.path.join(plate_train_labels, txt_name)
        pts = TXT_To_BBOX(lbl_path)
        cv2.polylines(img, [pts], True, (0, 255, 0), 5, cv2.LINE_AA)

        plt.imshow(img)
        plt.axis('off')
        i += 1
Plot_Random_Images()

# 4. Create a YoloV8n model

In [ ]:
# Create a YoloV8n model and load pretrained weights
yolo_model = YOLO('yolov8n.yaml').load('yolov8n.pt')

# Train the model with train images and labels
result = yolo_model.train(data=plate_yaml_path, epochs=20, amp=False)

# 5.Train Result

In [ ]:
# List of some metrics
metrics = ['results.png','confusion_matrix.png', 'P_curve.png', 'PR_curve.png']

plt.figure(figsize=(15, 12))
i = 1

for image_name in metrics:
    # Path of metrics images
    image_path = os.path.join(result_path, image_name)
    image = cv2.imread(image_path)

    # Plot images
    plt.subplot(2, 2, i)
    plt.imshow(image)
    plt.title(image_name)
    i += 1

plt.show()

In [ ]:
# Read csv of result
df = pd.read_csv(os.path.join(result_path, 'results.csv'))
df.tail(5)

In [ ]:
# Strip columns
df.columns = df.columns.str.strip()
df.columns

In [ ]:
# Define X, y to plot
X = df.epoch
cols = df.columns[1:]

plt.figure(figsize=(15, 25))

for it in range(len(cols)) :
    plt.subplot(7, 2, it+1)
    ax = sns.lineplot(data=df, x=X, y=cols[it])
    ax.set_title(cols[it])
plt.suptitle('Training Metrics and Loss', fontsize=24)
plt.subplots_adjust(top=0.6)
plt.tight_layout()
plt.show()

## 6.Best weights for training, stored in weights folder as best.pt. load best weight in a model.

In [ ]:
# Path of working-Directory
working_dir = 'C:\\envs\\Working\\Iranian_Plates'

In [ ]:
# Load model with best weights
best_model_path = os.path.join(result_path, 'weights/best.pt')
best_model_plate = YOLO(best_model_path)

In [ ]:
# Evaluating the model
result_test = best_model_plate.val()

In [ ]:
# list of metrics
keys = ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']

for key in keys :
    print(colored(f'{key} : {result_test.results_dict[key]}', 'green', attrs=['bold']))

## 7.Test the detection model

In [ ]:
# all test images
images = os.listdir(plate_test_images)

# K - number of random images
num_samples = 12
random_images = random.choices(images, k=num_samples)

# Create a list of random images paths
images = []
for image in random_images :
    image_path = os.path.join(plate_test_images, image)
    images.append(image_path)

images

In [ ]:
# Predict bbox of images one by one
for image in images :
    result = best_model_plate.predict([image], save=True, conf=0.5, iou=0.7)

In [ ]:
result_path = result[0].save_dir
print(result_path)

## 8.And plot the car images with predicted bboxes. Its a visual test

In [ ]:
# Show images
i = 1
plt.figure(figsize=(12, 6), dpi=200)
for image in os.listdir(result_path) :
    image_path = os.path.join(result_path, image)
    plt.suptitle('Test Result', fontsize=25, fontweight=500)
    plt.subplot(2, 5, i)
    plt.imshow(plt.imread(image_path))
    plt.axis('off')
    plt.subplots_adjust(top=0.75)
    i+=1
    if i==11 : break
plt.show()